In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!sudo apt install -y fluidsynth

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [3]:
!pip install --upgrade pyfluidsynth

In [4]:
!pip install pretty_midi
!pip install pydub
!pip install midi2audio
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [5]:
import collections
import numpy as np
import pandas as pd
import pretty_midi
import glob
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

In [6]:
# Function to convert MIDI files to notes
def midi_to_notes(midi_file: str) -> pd.DataFrame:
    pm = pretty_midi.PrettyMIDI(midi_file)
    instrument = pm.instruments[0]
    notes = collections.defaultdict(list)

    # Sort the notes by start time
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start

    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - prev_start)
        notes['duration'].append(end - start)
        prev_start = start

    return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

# Directory containing MIDI files
midi_dir = '/content/drive/MyDrive/midi/maestro-v2.0.0/**/*.midi'
midi_files = glob.glob(midi_dir, recursive=True)

In [10]:
len(midi_files)

1282

In [7]:
midi_files = midi_files[:len(midi_files)//5]
print(f"length of midi files: {len(midi_files)}")

length of midi files: 256


In [8]:
all_notes_df = pd.concat([midi_to_notes(f) for f in midi_files if not midi_to_notes(f).empty], axis=0)

In [9]:
all_notes_df

,pitch,start,end,step,duration
0,56,1.087240,1.299479,0.000000,0.212240
1,63,1.087240,1.299479,0.000000,0.212240
2,60,1.089844,1.290365,0.002604,0.200521
3,68,1.440104,1.584635,0.350260,0.144531
4,67,1.554688,1.683594,0.114583,0.128906
...,...,...,...,...,...
12532,33,802.628906,802.736979,0.001302,0.108073
12533,93,802.631510,802.666667,0.002604,0.035156
12534,95,802.645833,802.662760,0.014323,0.016927
12535,40,802.665365,802.666667,0.019531,0.001302


In [10]:
all_notes_df.to_csv('/content/drive/MyDrive/all_notes_df.csv')

In [11]:
all_notes_df.head()

,pitch,start,end,step,duration
0,56,1.087240,1.299479,0.000000,0.212240
1,63,1.087240,1.299479,0.000000,0.212240
2,60,1.089844,1.290365,0.002604,0.200521
3,68,1.440104,1.584635,0.350260,0.144531
4,67,1.554688,1.683594,0.114583,0.128906


In [12]:
# Normalize and process notes
note_min, note_max = all_notes_df['pitch'].min(), all_notes_df['pitch'].max()
all_notes_df['pitch'] = (all_notes_df['pitch'] - note_min) / (note_max - note_min)
time_max = all_notes_df['end'].max()
all_notes_df['start'] /= time_max
all_notes_df['end'] /= time_max
all_notes_df['step'] /= time_max
all_notes_df['duration'] /= time_max

all_notes = all_notes_df.to_numpy()

# Prepare sequences for training
SEQ_LENGTH = 50  # Length of each input sequence
STEP = 1  # Step size for creating sequences

def create_sequences(notes, seq_length, step):
    inputs = []
    targets = []
    for i in range(0, len(notes) - seq_length, step):
        inputs.append(notes[i:i + seq_length])
        targets.append(notes[i + seq_length])  # Predict the next note
    return np.array(inputs), np.array(targets)

X, y = create_sequences(all_notes, SEQ_LENGTH, STEP)

# Reshape for LSTM
X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))
print("Input shape:", X.shape)
print("Target shape:", y.shape)

Input shape: (1273466, 50, 5)
Target shape: (1273466, 5)


In [14]:
# Define the LSTM Model
model = Sequential([
    LSTM(128, input_shape=(SEQ_LENGTH, X.shape[2]), return_sequences=True),
    LSTM(128),
    Dense(X.shape[2])  # Predicting all note parameters
])

model.compile(optimizer='adam', loss='mse')

# Train the Model
history = model.fit(X, y, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
15919/15919 [==============================] - 3653s 229ms/step - loss: 0.0039 - val_loss: 0.0040
Epoch 2/5
15919/15919 [==============================] - 3622s 228ms/step - loss: 0.0036 - val_loss: 0.0037
Epoch 3/5
15919/15919 [==============================] - 3698s 232ms/step - loss: 0.0035 - val_loss: 0.0037
Epoch 4/5
15919/15919 [==============================] - 3717s 234ms/step - loss: 0.0034 - val_loss: 0.0036
Epoch 5/5
15919/15919 [==============================] - 3613s 227ms/step - loss: 0.0034 - val_loss: 0.0036


In [16]:
model.save('/content/drive/MyDrive/LSTM_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
def generate_music(model, seed, length):
    generated = []
    current_seq = seed

    for _ in range(length):
        prediction = model.predict(current_seq[np.newaxis, :, :])[0]
        generated.append(prediction)
        new_note = prediction
        current_seq = np.concatenate([current_seq[1:], new_note[np.newaxis, :]], axis=0)

    return np.array(generated)

# Seed sequence
seed_seq = X[0]

# Generate music sequence (longer for 10 seconds)
generated_notes = generate_music(model, seed_seq, 10)

# Denormalize generated notes
generated_notes[:, 0] = generated_notes[:, 0] * (note_max - note_min) + note_min  # pitch
generated_notes[:, 1:5] = generated_notes[:, 1:5] * time_max  # start, end, step, duration

# Convert generated notes to MIDI
def notes_to_midi(generated_notes, file_name='generated_music.mid'):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    for note_value in generated_notes:
        pitch = int(note_value[0])
        start_time = note_value[1]
        end_time = note_value[2]
        velocity = 100  # Fixed velocity for simplicity
        midi_note = pretty_midi.Note(
            velocity=velocity,
            pitch=pitch,
            start=start_time,
            end=end_time
        )
        instrument.notes.append(midi_note)

    midi.instruments.append(instrument)
    midi.write(file_name)

# Save generated music
notes_to_midi(generated_notes, 'generated_music.mid')

1/1 [==============================] - 0s 33ms/step


In [18]:
from midi2audio import FluidSynth

def midi_to_wav(midi_file, wav_file):
    fs = FluidSynth()
    fs.midi_to_audio(midi_file, wav_file)


midi_to_wav('generated_music.mid', 'generated_music.wav')

In [21]:
import shutil

shutil.copyfile('generated_music.mid', '/content/drive/MyDrive/generated_music.mid')
shutil.copyfile('generated_music.wav', '/content/drive/MyDrive/generated_music.wav')

'/content/drive/MyDrive/generated_music.wav'